# Algoritmo - Localização Bilhetagem (fora terminais e BRT)
### _Autor: Antônio Claudio Dutra Batista_
### _Orientador: Francisco Moraes de Oliveria Neto_
### _Metodologia: Mesclagem por informações coincidentes e critério de tempo_

# 1.0  Leitura das Bases

## 1.1 leitura da base - Bilhetagem (Tratamento Inicial)

In [ ]:
# Arquivos são do mês de novembro de 2018.
import pandas as pd

# Definindo tipos de variáveis com tamanhos menores
tipos_specification = {'id': 'int32', 'linha': 'int32'}

# Lista para armazenar os dataframes de cada grupo de 10 dias
grupos_dias = []

# Loop para ler os arquivos CSV e concatená-los em grupos de 10 dias
for i in range(1, 31, 10):  # Início, fim (não inclusivo), passo
    dias = [pd.read_csv(f'2018-11-{str(j).zfill(2)}.csv', usecols=['id', 'linha', 'nome_linha', 'prefixo_carro', 'nome_cartao', 'sentido_viagem', 'dia', 'momento'], dtype=tipos_specification, sep=',') for j in range(i, i + 10)]
    grupo = pd.concat(dias)
    grupos_dias.append(grupo)

In [ ]:
# Concatenando os grupos de 10 dias
mes_tudo = pd.concat(grupos_dias)

In [ ]:
# BUSCAR DADOS NULOS
mes_tudo.info()

In [ ]:
# Momentos em ordem do df mes_c_R!
mes_tudo.sort_values('momento', inplace = True) # Mostrando DF em ORDEM!

In [ ]:
# Memoria liberou 0.2 gb
mes_tudo.info()

In [ ]:
# Juntando colunas de interesse (dia, momento) em uma só:
mes_tudo["momento"] = mes_tudo['dia'].astype(str) + ' ' + mes_tudo['momento'].astype(str)

In [ ]:
# REMOVENDO COLUNAS DESNECESSÁRIAS!
mes1 = mes_tudo.drop(['dia'], axis=1)

In [ ]:
# Excluindo explicitamente variavel com bilhetagem desnecessária
del mes_tudo

In [ ]:
mes1.info()

In [ ]:
mes1["momento"] = pd.to_datetime(mes1["momento"])

In [ ]:
# Removendo dados nulos na coluna prefixo_carro
mes_ = mes1.dropna(subset = ['prefixo_carro'])

In [ ]:
# Excluindo explicitamente variavel com bilhetagem desnecessária
del mes1

In [ ]:
# Removendo identificação de veiculos com caracteris da coluna de prefixo_carro
mes = mes_[pd.to_numeric(mes_['prefixo_carro'], errors='coerce').notnull()]

In [ ]:
# Trasnformando coluna de prefixo carro em inteiro
mes['prefixo_carro'] = mes['prefixo_carro'].astype('int32')

In [ ]:
mes.info()

In [ ]:
# Última versão do df MES
mes

## 1.2 leitura da base - Dicionário

In [ ]:
# df', abaixo:
d1 = pd.read_csv('veiculos2018.csv', sep= ';')

In [ ]:
# Mostrando Df
display(d1)

In [ ]:
d1.info()

In [ ]:
# transformando coluna para tipo string
d1['cod_veiculo'] = d1['cod_veiculo'].astype(str)

In [ ]:
# Removendo casracters não numericos da calouna
d1['cod_veiculo']= d1['cod_veiculo'].replace({'\D': ''}, regex=True)

In [ ]:
# Convertendo dados para valores numericos
d1['cod_veiculo'] = pd.to_numeric(d1['cod_veiculo'], errors='coerce')

In [ ]:
d1

In [ ]:
# Removenfdo identificação de veiculos com caracteres da coluna de cod_veiculo
# d01 = d1[pd.to_numeric(d1['cod_veiculo'], errors='coerce').notnull()]

In [ ]:
d1.info()

In [1]:
''' Como algumas informações da coluna cod_veiculo possuiam caracteres em todas a celula, esses valores foram removidos por interio, então as linhas dessas colunas ficaram com dados nulos! '''

' Como algumas informações da coluna cod_veiculo possuiam caracteres em todas a celula, esses valores foram removidos por interio, então as linhas dessas colunas ficaram com dados nulos! '

In [ ]:
# Verificando dados nulos 
nulos_cod = d1[d1['cod_veiculo'].isnull()]
len(nulos_cod)

In [ ]:
# Removendo dados nulos do dicionario
d01 = d1.dropna()

In [ ]:
# Trasnformando coluna de prefixo carro em inteiro
d01['cod_veiculo'] = d01['cod_veiculo'].astype('int32')
d01['id_veiculo'] = d01['id_veiculo'].astype('int32')

In [ ]:
d01[d01.cod_veiculo ==36981]

In [ ]:
d01['cod_veiculo'].sort_index()

In [ ]:
d01.isnull().sum()

## 1.3 leitura do base - GPS (Tratamento INICIAL)

In [ ]:
import pandas as pd

# Definindo o número de linhas que você deseja ler
numero_de_linhas_para_ler = 45000000  # Substitua pelo número desejado
dtype_specification = {'longitude': 'float32', 'latitude': 'float32', 'vehicle_vehicleid': 'int32'}

# Usando o pandas para ler o CSV, limitando o número de linhas com a opção nrows
mes_ate_45_milhoes = pd.read_csv('Paint112018.csv',usecols=['latitude', 'longitude', 'metrictimestamp', 'vehicle_vehicleid'], dtype=dtype_specification, nrows=numero_de_linhas_para_ler)

In [ ]:
# Lendo restante do arquivo
mes_resto = pd.read_csv('Paint112018.csv', usecols=['latitude', 'longitude','metrictimestamp', 'vehicle_vehicleid'],dtype=dtype_specification, skiprows=range(1, numero_de_linhas_para_ler + 1))

In [ ]:
# Juntando em um mesmo df
GPS_I_H =  pd.concat([mes_ate_45_milhoes, mes_resto]) 

In [ ]:
# Excluindo explicitamente variavel com dados do GPS separados
del mes_ate_45_milhoes
del mes_resto

In [ ]:
display(GPS_I_H.iloc[275])

In [ ]:
GPS_I_H.info()

In [ ]:
# FORMATANDO metrictimestamp do (GPS)
GPS_I_H['momento'] = pd.to_datetime(GPS_I_H['metrictimestamp'],format= '%Y%m%d%H%M%S')

In [ ]:
GPS_I_H.drop("metrictimestamp", axis= 1, inplace= True)

In [ ]:
# Renomear coluna de momento de vehicle_vehicleid para id_veiculo:
GPS_I_H.columns=['latitude', 'longitude','id_veiculo', 'momento']

In [ ]:
# Colocando valores em ordem
GPS_I_H.sort_values('momento', inplace= True)

# 2.0 Junção da bilhetagem com o dicionário 

In [ ]:
# Acrescentando dicionário
mes_d1_T = mes.merge(d01, left_on= 'prefixo_carro', right_on='cod_veiculo')

In [ ]:
mes_d1 = mes_d1_T

In [ ]:
del mes_d1_T

In [ ]:
# Base da Bilhetagem adicionado o dicionario
mes_d1

In [ ]:
mes_d1.info()

In [ ]:
# Criando df somente colunas necessátrias para a integração no tópico 03!
mes_geo = mes_d1.sort_values('momento')

In [ ]:
del mes_d1

In [ ]:
# Reiniciando a contagem dos index
mes_geo.reset_index(inplace=True)

In [ ]:
# REMOVENDO COLUNA DESNECESSÁRIA!
mes_georr = mes_geo.drop(['index', "prefixo_carro"], axis=1)

In [ ]:
mes_georr.info()

In [ ]:
del mes_geo

# 3.0 Integração da coordenada aproximada do ônibus na bilhetagem

In [ ]:
# Coordenada mais proxima do ônibus é extraída da base de GPS e integrada na base da bilhetagem (SOMENTE COLUMAS MOMENTO E ID)
mes_d1_GPS = pd.merge_asof(mes_georr, GPS_I_H, left_on='momento', right_on='momento', by='id_veiculo', direction ='nearest')

In [ ]:
mes_d1_GPS.isnull().sum()

In [ ]:
# Base da bilhetagem localizada 
mes_d1_GPS

# 4.0 Plotagem da bilhetagem georreferenciada


## 4.1 Intalação das bibliotecas e importação
''' OBS: Os codigos abaixo serão deixados no arquivo, contudo o objetivo do procedimento não é visualização dos dados de bilhetagem localizados mas serão deixados em caso que se deseja utilizar essas bibliotecas para esse fim! '''

In [ ]:
# INSTALANDO A BIBLIOTECA GEOPANDAS COM CONDA
# !conda install geopandas -y

# instalando com pip
!pip install-q geopandas

In [ ]:
# mostrar a versão da biblioteca 
# !pip list geopandas

In [ ]:
# instalando a biblioteca matplotlib com conda
# !conda install matplotlib -y

# instalando com pip
!pip install -q matplotlib

In [ ]:
# mostrar a versão da biblioteca 
# !pip list matplotlib

In [ ]:
# importando as bibliotecas
import geopandas as gpd
import matplotlib.pyplot as plt

## 4.2 Leitura de dados espaciais 

In [ ]:
# Ler os dados com geopandas
ceara_muni = gpd.read_file('CE_Municipios_2022.shp')

### 4.2.1 Visualização dos dados 

In [ ]:
# Visualização
# ceara_muni.plot(figsize=(16,14), facecolor='white', edgecolor='black') 
#plt.show()

### 4.2.2 Filtro somente da capital 

In [ ]:
# Salvando somente a capital 'Fortaleza'
fortal = ceara_muni[ceara_muni.NM_MUN=='Fortaleza']
fortal_p = fortal.iloc[0].geometry

In [ ]:
# Mostrar capital 
fortal_p

In [ ]:
# Salvar o geodataframe com formato geojson
filename =  'fortal.json'
fortal.to_file(filename, driver='GeoJSON')

In [ ]:
fortall = gpd.read_file(filename, driver='GeoJSON')

In [ ]:
fortall

### 4.2.3 Modelo de dados espaciais e criação de coluna geometrica 

In [ ]:
# Instalando biblioteca shapely com pip
!pip install -q shapely
# Mostrar versão da biblioteca 
# !pip list shapely

In [ ]:
# importando os objetos geométricos da biblioteca shapely
from shapely.geometry import Point, LineString, Polygon

In [ ]:
# Criando coluna geometerica no df (mes_d1_GPS)
# mes_d1_GPS['geometry'] = None 
# for index, row in mes_d1_GPS.iterrows():
    #mes_d1_GPS.loc[index, 'geometry'] = Point(row.latitude, row.longitude)

In [ ]:
# mes_d1_GPS

In [ ]:
# Criando coluna geometerica no df (mes_d1_GPS)
# mes_d1_GPS['geometry'] = mes_d1_GPS.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [ ]:
# tipos de dados 
# type(mes_d1_GPS)

In [ ]:
# Criando um GeoDataFrame 
# mes_d1_GPS_geodat_ = gpd.GeoDataFrame(mes_d1_GPS, geometry='geometry')

In [ ]:
# mostar dados
# mes_d1_GPS_geodat_= mes_d1_GPS

In [ ]:
# Salvar o geodataframe com formato geojson
# filename2 =  'mes_d1_GPS_geodat_.json'
# mes_d1_GPS_geodat_.to_file(filename2, driver='GeoJSON')

In [ ]:
# mes_d1_GPS_geodat = gpd.read_file(filename2, driver='GeoJSON')

In [ ]:
# plotando dados (visualização)
# mes_d1_GPS_geodat.plot(figsize=(8,8), facecolor='white', edgecolor='black')

In [ ]:
# type(mes_d1_GPS_geodat)

### 4.2.4 Operação de interseção entre geometrias 

In [ ]:
# Mostrar os dados simultaneamente
# fig, ax = plt.subplots(figsize=(15,15))

# mes_d1_GPS_geodat.plot(ax=ax)
# fortall.plot(ax=ax, facecolor='None', edgecolor='black')

In [ ]:
# VERIFICAMOS QUE EXISTEM DADOS GEORREFERENCIADOS FORA DA MALHA DA CIDADE, PODENDO SER NA SUA MAIORIA REGISTRO DOS VEICULOS
# QUANDO SE DESLOCARAM PARA OUTRAS ZONAS, OU ATÉ MESMO ERROS DO APARELHO DE GPS, COMO MOSTRA NA FIGURA ACIMA UM REGISTRO 
# PRATICAMENTE NO MAR.

In [ ]:
# Realizar operação de interseção entre geometrias
# mes_geo_fortal = mes_d1_GPS_geodat[mes_d1_GPS_geodat.intersects(fortal_p)]

In [ ]:
# mes_geo_fortal.info()

In [ ]:
# NOTAMOS QUE DOS 891038 DADOS DE VALIDAÇÕES APENAS 825386 foram localizadas dentro da malha da cidade, seguimos com estas!

In [ ]:
# Mostrar os dados simultaneamente após interseção
# fig, ax = plt.subplots(figsize=(10,10))

# mes_geo_fortal.plot(ax=ax)
# fortall.plot(ax=ax, facecolor='None', edgecolor='black')

In [ ]:
# Salvando os dados georreferenciados em fortaleza 
# filename3 = 'mes_geo_fortal.json'

# mes_geo_fortal.to_file(filename3, driver='GeoJSON')

### 4.2.5 Mostrando dados no mapa de agrupamento (clusters)

In [ ]:
# Instalando biblioteca folium com pip!
!pip install -q folium

In [ ]:
# importando biblioteca folium e pluging
import folium 
from folium.plugins import FastMarkerCluster

In [ ]:
# Criar mapa de agrupamentos (clusters)
# media_long = mes_geo_fortal['longitude'].mean()
# media_lat = mes_geo_fortal['latitude'].mean()

# mapa = folium.Map(location=[media_lat, media_long])

In [ ]:
# Transformando dados em uma feat geojson
# mc = FastMarkerCluster(mes_geo_fortal[['latitude', 'longitude']])

In [ ]:
# Poligonal da cidade no mapa
limites = folium.features.GeoJson(fortall, style_function=lambda feature:{'color': 'black', 'weight': 2, 'fillOpacity':0.0})

In [ ]:
# Adicionando dados no mapa
# mapa.add_child(mc)
# mapa.add_child(limites)       

In [ ]:
# Salvar o nosso mapa em um arquivo html
# mapa.save('validações_georreferenciadas.html')

### 4.2.6 Mostrando dados no mapa de calor (Heatmap)

In [ ]:
# Importando a plugin HeatMap
from folium.plugins import HeatMap

In [ ]:
# Criando mapa II

# mapa_2 = folium.Map(location=[media_lat, media_long]) 

# Criando heat_map
# heat_map = HeatMap(mes_geo_fortal[['latitude', 'longitude']])     

In [ ]:
# Adicionando no map
# mapa_2.add_child(heat_map)
# mapa_2.add_child(limites)

In [ ]:
# Salvar o nosso mapa em um arquivo html
# mapa_2.save('validações_georreferenciadas_Heatmap_calor.html')

# 5.0 Testes

## 5.1.0 Motivo do não georreferenciamento da base completa da bilhetagem

In [ ]:
# Inspeção na base da bilhetagem e do dicionario para um veiculo especifico
veiculo_12994 = mes[mes.prefixo_carro==12994]

In [ ]:
# NOTAMOS QUE ESSE VEICULO ESPECIFICO POSSUI REGISTROS NA BILHETAGEM

In [ ]:
### Verificando dados que estão presentes na base da bilhetagem e não na base do dicionario
lista = d01.cod_veiculo.unique()
mes.isin({'prefixo_carro':lista}).prefixo_carro.value_counts(1)

In [ ]:
# OU SEJA, DOS 100% DE DADOS NA BILHETAGEM APENAS 86,78% POSSUI UM CODIGO CORRESPONDENTE NA BASE DO DICIONARIO!

In [ ]:
# VERIFICAÇÃO DO VEICULO 12994 NA BASE DO DICIONARIO 
d01[d01.cod_veiculo == 12993]

In [2]:
''' CONCLUSÃO, APÓS INSPEÇÃO NOTAMOS QUE NÃO FOI POSSIVEL GEORREFERENCIAR A BASE DA BILHETAGEM COMPLETA POIS PARA ALGUNS VEICULOS NÃO TEM UM CODIGO DE VEICULO CORRESPONDENTE NA BASE DO DICIONARIO E CONSEQUENTIMENTE NÃO VAI SER LOCALIZADO NA BASE DO GPS, ASSIM UMA PERCENTAGEM DOS DADOS DA BILHETAGEM NÃO TIVERAM COORDENADA APROXIMADA DE VALIDAÇÃO GEORREFERENCIADA e o metodo aplicado considerou a remoção de dados com caracters do dicionario deixanod apenas dados numericos! '''

' CONCLUSÃO, APÓS INSPEÇÃO NOTAMOS QUE NÃO FOI POSSIVEL GEORREFERENCIAR A BASE DA BILHETAGEM COMPLETA POIS PARA ALGUNS VEICULOS NÃO TEM UM CODIGO DE VEICULO CORRESPONDENTE NA BASE DO DICIONARIO E CONSEQUENTIMENTE NÃO VAI SER LOCALIZADO NA BASE DO GPS, ASSIM UMA PERCENTAGEM DOS DADOS DA BILHETAGEM NÃO TIVERAM COORDENADA APROXIMADA DE VALIDAÇÃO GEORREFERENCIADA e o metodo aplicado considerou a remoção de dados com caracters do dicionario deixanod apenas dados numericos! '

# ------------------------------------------------------------------------